In [1]:
import pandas as pd
import os
from datetime import date, datetime
import numpy as np

C:\Users\marie\AppData\Local\Temp\ipykernel_36772\1199149209.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [22]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "..", "data")
df = pd.read_csv(os.path.join(data_path, "merged_lactation.csv"))


## find the teats that did not produce milk

In [23]:
list_temp = []
thres = 5
for cow in df.animal_number:
    cow_line = df[df.animal_number == cow]
    lfy = cow_line["yield_lf_visit_sum"].values[0]
    rfy = cow_line["yield_rf_visit_sum"].values[0]
    lby = cow_line["yield_lr_visit_sum"].values[0]
    rby = cow_line["yield_rr_visit_sum"].values[0]
    if lfy < thres:
        list_temp.append([cow, 'lf', lfy])
    if rfy < thres:
        list_temp.append([cow, 'rf', rfy])
    if lby < thres:
        list_temp.append([cow, 'rr', lby])
    if rby < thres:
        list_temp.append([cow, 'lr',rby])

missing_teat_df = pd.DataFrame(list_temp, columns = ["cow", "teat", 'y'])
missing_teat_df.to_csv(os.path.join(data_path,"missing_teats.csv"), index = False)

## change values from missing teats to NA

In [33]:
df2 = df.copy()
cows_missing = np.unique(missing_teat_df.cow)

for cow in cows_missing:
    teats_missing = missing_teat_df[missing_teat_df.cow == cow].reset_index()["teat"]
    for teat in teats_missing:
        columns_na = [col for col in df.columns if teat in col]
        idx = np.where(df2.animal_number == cow)[0]
        df2.loc[idx, columns_na] = np.nan

df2.to_csv(os.path.join(data_path,"merged2_lactation.csv"), index = False)

In [32]:
df2

,animal_number,min_date,max_date,count_date,min_dim,max_dim,milk_flow_duration_mean,yield_visit_mean,yield_lf_visit_mean,yield_lr_visit_mean,...,kickoff_rf_perc,kickoff_rr_perc,kickoff_front_perc,kickoff_back_perc,kickoff_any_perc,yield_lf_visit_sum,yield_lr_visit_sum,yield_rr_visit_sum,yield_rf_visit_sum,interval_visit_mean
0,454,2023-03-21,2023-11-17,246,0.0,241.0,381.243711,18.092107,3.775346,5.109465,...,0.460829,2.764977,1.228879,7.834101,8.141321,2432.15,3276.69,3782.15,2131.24,31594.316436
1,571,2023-10-20,2023-11-17,29,0.0,28.0,450.400000,17.547333,3.515167,6.301667,...,7.812500,20.312500,10.937500,32.812500,35.937500,225.47,392.54,309.63,175.25,37965.250000
2,627,2023-08-17,2023-11-17,94,0.0,92.0,244.504032,19.376452,3.784556,5.401331,...,0.000000,0.000000,0.000000,0.000000,0.000000,948.77,1347.54,1480.53,1065.06,31442.340000
3,628,2023-10-17,2023-11-17,32,0.0,31.0,332.936709,16.635190,4.160886,6.386076,...,NaN,1.265823,0.000000,1.265823,1.265823,328.71,504.50,480.97,NaN,33534.810127
4,686,2023-09-28,2024-09-23,365,0.0,361.0,316.455366,20.467924,4.461334,5.744544,...,0.098425,0.590551,0.196850,1.476378,1.574803,4501.31,5808.03,5664.09,4713.77,30266.279528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,1501,2023-10-27,2023-11-17,22,0.0,21.0,263.775000,13.026500,3.134250,3.378250,...,30.232558,51.162791,44.186047,62.790698,62.790698,134.60,146.79,132.51,142.79,41876.093023
235,1503,2023-11-10,2024-10-22,354,0.0,347.0,319.529412,16.081324,3.855027,4.023636,...,0.658762,0.527009,1.712780,1.054018,1.712780,2903.74,3052.96,3131.90,3077.57,39046.085639
236,1504,2023-11-06,2024-08-20,292,1.0,289.0,425.970423,13.171338,2.779789,3.485859,...,0.273973,0.410959,0.410959,0.547945,0.821918,1993.32,2526.68,2512.82,2507.76,33584.263014
237,1507,2023-11-10,2023-11-17,8,0.0,7.0,318.642857,5.433571,0.634286,0.132857,...,0.000000,NaN,0.000000,13.333333,13.333333,9.17,1.03,NaN,32.75,40166.733333
